# Exploratory Data Analysis (EDA) – European Soccer Dataset

This notebook performs exploratory data analysis (EDA) on the **European Soccer Database** (`database.sqlite`).

Goals of this EDA:

- Understand the **structure and size** of the relational database  
- Inspect the **target distribution** (Win / Draw / Loss)  
- Analyze **league representation** (for fairness considerations)  
- Study **missing data patterns** (to justify feature exclusions)  
- Explore **player attributes** distributions and correlations  
- Detect basic **temporal drift** over seasons  

Outputs:

- Plots saved in `results/EDA/`  
- Summary tables saved as `.csv` in `results/EDA/`  

> **Note:** This notebook is for exploration only.  
> The production feature engineering and modeling pipelines live in `src/predictiq/` and are used by other notebooks.

### Data Loading Note
This notebook uses the shared project library (`predictiq.data`) to load the
Kaggle European Soccer database:

```python
from predictiq.data import load_db
conn, q = load_db()
```

This ensures consistent, reproducible data access across all notebooks and scripts.

All figures and tables generated in this notebook are saved under:
`results/EDA/`  

These files are used later in the final report and model card.


In [1]:
"""
EDA on the European Soccer Database:
- DB structure, class balance, league representation
- Missingness, player attributes, temporal drift
- Bookmaker odds as an external baseline
"""

# Install extra libs only if running in a fresh environment
# !pip install seaborn sqlalchemy tqdm kagglehub --quiet




import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from predictiq.data import load_db

# Display settings
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

sns.set_theme(style="whitegrid")
plt.rcParams.update({
    "figure.figsize": (8, 5),
    "axes.titlesize": 18,
    "axes.labelsize": 14,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    "axes.spines.top": False,
    "axes.spines.right": False,
})

PRIMARY_COLOR = "#AA336A"
SECONDARY_COLOR = "#ff7f0e"

# Create results directory
os.makedirs("results/EDA", exist_ok=True)
print("Created directory: results/EDA")


ModuleNotFoundError: No module named 'predictiq'

In [ ]:
# Use shared data loader from the predictiq package
conn, q = load_db()

## 1. Database Structure – Tables and Row Counts

We first inspect the list of tables and their row counts to understand:

- Where most of the data volume is  
- Which tables are central for modeling (e.g., `Match`, `Player_Attributes`)  
- Whether the dataset size is feasible for our ML pipelines  

In [ ]:
# List tables
tables = q("""
SELECT name
FROM sqlite_master
WHERE type='table' AND name NOT LIKE 'sqlite_%'
ORDER BY name
""")
tables


In [ ]:
# Row counts per table
row_counts = []
for t in tables["name"]:
    n = q(f"SELECT COUNT(*) AS n FROM {t}")["n"].iloc[0]
    row_counts.append({"table": t, "rows": n})

row_counts_df = pd.DataFrame(row_counts).sort_values("rows", ascending=False)
row_counts_df


In [ ]:
# Save to CSV for GitHub
row_counts_df.to_csv("results/EDA/row_counts.csv", index=False)
print("Saved table row counts to results/EDA/row_counts.csv")


In [ ]:
list(q("PRAGMA table_info(Match)")["name"])


## 2. Match-Level View and Target Definition

We focus on the `Match` table and construct our target variable:

- `match_result ∈ {home_win, draw, away_win}` based on final score  
- Convert `date` to a proper datetime  
- This defines the classification problem for the project.  


In [ ]:
matches = q("""
SELECT
    id, match_api_id, country_id, league_id, season, stage, date,
    home_team_api_id, away_team_api_id,
    home_team_goal, away_team_goal,
    home_player_1, home_player_2, home_player_3, home_player_4, home_player_5,
    home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11,
    away_player_1, away_player_2, away_player_3, away_player_4, away_player_5,
    away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11,
    B365H, B365D, B365A
FROM Match
""")

matches["date"] = pd.to_datetime(matches["date"], errors="coerce")

def result_label(row):
    if row["home_team_goal"] > row["away_team_goal"]:
        return "home_win"
    elif row["home_team_goal"] < row["away_team_goal"]:
        return "away_win"
    else:
        return "draw"

matches["match_result"] = matches.apply(result_label, axis=1)

matches.head()


## 3. Match Result Distribution (Win / Draw / Loss)

We inspect the distribution of outcomes:

- To understand **class balance**  
- To justify using **Macro-F1** instead of plain accuracy  
- To quantify the home advantage  


In [ ]:
result_counts = matches["match_result"].value_counts().reindex(["home_win","draw","away_win"])
result_share = result_counts / result_counts.sum()

print("Raw counts:")
print(result_counts)
print("\nClass distribution ratio:")
print(result_share.round(3))


In [ ]:
plt.figure(figsize=(6,4))
ax = sns.barplot(x=result_counts.index, y=result_counts.values)
plt.title("Match Outcome Distribution (All Seasons)")
plt.xlabel("Result")
plt.ylabel("Number of Matches")

for i, v in enumerate(result_counts.values):
    ax.text(i, v + 200, f"{result_share.values[i]*100:.1f}%",
            ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig("results/EDA/match_result_distribution.png", dpi=300)
plt.show()
plt.close()

print("Saved plot: results/EDA/match_result_distribution.png")


## 4. League Representation and Imbalance

We join `Match` with `League` and `Country` to see:

- How many matches belong to each league  
- Whether some leagues dominate the dataset  
- This informs our **fairness audits** and per-league evaluation.  


In [ ]:
list(q("PRAGMA table_info(league)")["name"])

In [ ]:
list(q("PRAGMA table_info(country)")["name"])

In [ ]:
leagues = q("SELECT * FROM League")
countries = q("SELECT * FROM Country").rename(columns={"name": "country_name"})

m_league = matches.merge(leagues, left_on="league_id", right_on="id", suffixes=("", "_league"))
m_league = m_league.merge(countries, left_on="country_id", right_on="id", suffixes=("", "_country"))

m_league = m_league.rename(columns={"name": "league_name"})

league_counts = (m_league.groupby(["country_name", "league_name"])["match_api_id"]
                 .count()
                 .reset_index()
                 .rename(columns={"match_api_id": "match_count"})
                 .sort_values("match_count", ascending=False))

league_counts.head(15)


In [ ]:
# Save table
league_counts.to_csv("results/EDA/league_counts.csv", index=False)
print("Saved league counts to results/EDA/league_counts.csv")


In [ ]:
# Top 10 leagues barplot
total_matches = league_counts["match_count"].sum()
top10 = league_counts.head(10).copy()
top10["share"] = top10["match_count"] / total_matches

plt.figure(figsize=(8,5))
ax = sns.barplot(y="league_name", x="match_count", data=top10)
plt.title("Top 10 Leagues by Match Count")
plt.xlabel("Number of Matches")
plt.ylabel("League")

for i, row in top10.reset_index(drop=True).iterrows():
    ax.text(row["match_count"] + 50, i, f"{row['share']*100:.1f}%",
            va='center', fontsize=8)

plt.tight_layout()
plt.savefig("results/EDA/league_match_counts_top10.png", dpi=300)
plt.show()
plt.close()

print("Saved plot: results/EDA/league_match_counts_top10.png")


## 5. Missing Data Patterns (Match Table)

We analyze missing data in key match-level columns:

- Line-ups (`home_player_X`, `away_player_X`)  
- Betting odds (`B365H`, `B365D`, `B365A`)  

This justifies:

- Excluding line-ups and betting odds from the **first modeling version**  
- Handling missingness in a principled way.  


In [ ]:
key_cols = [
    "home_team_goal","away_team_goal",
    "home_team_api_id","away_team_api_id",
    "home_player_1","home_player_2","home_player_3","home_player_4","home_player_5",
    "home_player_6","home_player_7","home_player_8","home_player_9","home_player_10","home_player_11",
    "away_player_1","away_player_2","away_player_3","away_player_4","away_player_5",
    "away_player_6","away_player_7","away_player_8","away_player_9","away_player_10","away_player_11",
    "B365H","B365D","B365A"
]
key_cols = [c for c in key_cols if c in matches.columns]

miss_rate = matches[key_cols].isna().mean().sort_values(ascending=False)
miss_rate.head(20)


In [ ]:
# Save missing rates
miss_rate.to_csv("results/EDA/match_missing_rate.csv")
print("Saved missing rate table to results/EDA/match_missing_rate.csv")

# Barplot of missing rates
plt.figure(figsize=(10,4))
sns.barplot(x=miss_rate.index, y=miss_rate.values)
plt.xticks(rotation=90)
plt.ylabel("Missing Rate")
plt.title("Missing Rate per Key Match Column")
plt.tight_layout()
plt.savefig("results/EDA/missing_rate_barplot.png", dpi=300)
plt.show()
plt.close()

print("Saved plot: results/EDA/missing_rate_barplot.png")


In [ ]:
# Heatmap (sampled rows & top 15 columns)
top_missing_cols = miss_rate.head(15).index
sample = matches[top_missing_cols].sample(min(1000, len(matches)), random_state=42)

plt.figure(figsize=(8,10))
sns.heatmap(sample.isna(), cbar=True)
plt.title("Missingness Heatmap (Sampled Matches, Top 15 Columns)")
plt.xlabel("Columns")
plt.ylabel("Sampled Matches")
plt.tight_layout()
plt.savefig("results/EDA/missingness_heatmap_top15.png", dpi=300)
plt.show()
plt.close()

print("Saved plot: results/EDA/missingness_heatmap_top15.png")


## 6. Player Attributes – Distribution & Temporal Drift

We now inspect the `Player_Attributes` table:

- Distribution of `overall_rating`  
- Correlations between key skills  
- Temporal drift in player ratings over years  

This helps us understand the **feature space** for modeling.  


In [ ]:
list(q("PRAGMA table_info(Player_Attributes)")["name"])

In [ ]:
player_full = q("SELECT * FROM Player_Attributes")
player_full["date"] = pd.to_datetime(player_full["date"], errors="coerce")
missing_ratio = player_full.isna().mean().sort_values(ascending=False)
missing_ratio


In [ ]:
player_attrs = q("""
SELECT
    player_api_id, date,
    overall_rating, potential, finishing, dribbling, crossing, long_shots
FROM Player_Attributes
""")

player_attrs["date"] = pd.to_datetime(player_attrs["date"], errors="coerce")
player_attrs.head()


In [ ]:
data = player_attrs["overall_rating"].dropna()

print("overall_rating summary:")
print(data.describe())

plt.figure(figsize=(7,4))
sns.histplot(data, bins=30, kde=True)
plt.axvline(data.mean(), color='red', linestyle='--', label=f"Mean={data.mean():.1f}")
plt.axvline(data.median(), color='green', linestyle=':', label=f"Median={data.median():.1f}")
plt.title("Distribution of Player Overall Rating")
plt.xlabel("Overall Rating")
plt.ylabel("Count")
plt.legend()
plt.tight_layout()
plt.savefig("results/EDA/player_overall_rating_histogram.png", dpi=300)
plt.show()
plt.close()

print("Saved plot: results/EDA/player_overall_rating_histogram.png")


In [ ]:
player_attrs["year"] = player_attrs["date"].dt.year

rating_by_year = (player_attrs
                  .dropna(subset=["overall_rating", "year"])
                  .groupby("year")["overall_rating"]
                  .mean()
                  .reset_index())

rating_by_year


In [ ]:
rating_by_year.to_csv("results/EDA/drift_player_overall_rating_by_year.csv", index=False)
print("Saved drift table to results/EDA/drift_player_overall_rating_by_year.csv")

plt.figure(figsize=(7,4))
ax = sns.lineplot(data=rating_by_year, x="year", y="overall_rating", marker="o")
plt.title("Average Player Overall Rating by Year")
plt.xlabel("Year")
plt.ylabel("Mean Overall Rating")

for x, y in zip(rating_by_year["year"], rating_by_year["overall_rating"]):
    ax.text(x, y + 0.1, f"{y:.1f}", ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig("results/EDA/player_overall_rating_drift.png", dpi=300)
plt.show()
plt.close()

print("Saved plot: results/EDA/player_overall_rating_drift.png")


In [ ]:
num_cols = ["overall_rating", "potential", "finishing", "dribbling", "crossing", "long_shots"]
corr = player_attrs[num_cols].dropna().corr()
corr


In [ ]:
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1, annot_kws={"size": 8})
plt.title("Correlation Between Key Player Attributes")
plt.tight_layout()
plt.savefig("results/EDA/player_attribute_correlation.png", dpi=300)
plt.show()
plt.close()

print("Saved plot: results/EDA/player_attribute_correlation.png")


## 7. Season Coverage – Number of Matches per Season

We inspect how many matches are available in each season.  
This helps us understand:

- whether some seasons are under-represented,
- and supports our choice of chronological splits for training and evaluation.


In [ ]:
season_counts = (
    matches.groupby("season")["match_api_id"]
    .count()
    .reset_index()
    .rename(columns={"match_api_id": "match_count"})
    .sort_values("season")
)

season_counts


In [ ]:
fig, ax = plt.subplots(figsize=(9,4))

bars = ax.bar(season_counts["season"], season_counts["match_count"], color=PRIMARY_COLOR)

ax.set_title("Number of Matches per Season")
ax.set_xlabel("Season")
ax.set_ylabel("Number of Matches")

plt.xticks(rotation=45, ha="right")

plt.tight_layout()
plt.savefig("results/EDA/matches_per_season_presentation.png", dpi=300)
plt.show()


## 8. Home Advantage – Average Goals Scored

We quantify home advantage by comparing:

- average goals scored by home teams,
- average goals scored by away teams.

This supports our baseline reasoning (home win is the most common outcome).


In [ ]:
home_mean_goals = matches["home_team_goal"].mean()
away_mean_goals = matches["away_team_goal"].mean()

print("Average home goals:", round(home_mean_goals, 3))
print("Average away goals:", round(away_mean_goals, 3))

fig, ax = plt.subplots(figsize=(6,4))

labels = ["Home Goals", "Away Goals"]
values = [home_mean_goals, away_mean_goals]

bars = ax.bar(labels, values, color=[PRIMARY_COLOR, SECONDARY_COLOR])

ax.set_title("Average Goals Scored – Home vs Away")
ax.set_ylabel("Average Goals per Match")

for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2, val + 0.02, f"{val:.2f}",
            ha="center", va="bottom", fontsize=11, fontweight="bold")

plt.tight_layout()
plt.savefig("results/EDA/home_vs_away_goals_presentation.png", dpi=300)
plt.show()


## 9. Betting Odds – Implied Probabilities (Baseline Signals)

We derive implied probabilities from B365 odds:

- Convert decimal odds to implied probabilities,
- Normalize them to sum to 1,
- Compute a simple *home advantage index*.

These features are useful for:

- understanding how bookmakers view each match,
- building an **industry baseline** to compare our models against.


In [ ]:
odds_cols = ["B365H", "B365D", "B365A"]
has_all_odds = matches[odds_cols].notna().all(axis=1)
matches_odds = matches.loc[has_all_odds].copy()

# convert odds to implied probabilities
inv_home = 1.0 / matches_odds["B365H"]
inv_draw = 1.0 / matches_odds["B365D"]
inv_away = 1.0 / matches_odds["B365A"]

inv_sum = inv_home + inv_draw + inv_away

matches_odds["b365_home_prob"] = inv_home / inv_sum
matches_odds["b365_draw_prob"] = inv_draw / inv_sum
matches_odds["b365_away_prob"] = inv_away / inv_sum

matches_odds["home_advantage_index"] = (
    matches_odds["b365_home_prob"] - matches_odds["b365_away_prob"]
)

matches_odds[[
    "B365H","B365D","B365A",
    "b365_home_prob","b365_draw_prob","b365_away_prob",
    "home_advantage_index","match_result"
]].head()


## 10. Temporal Drift – Home Win Implied Probability by Season

We check how the bookmakers' estimated probability of a home win changes across seasons.

This is a pre-match feature and illustrates:

- temporal drift in match difficulty and league balance,
- additional motivation for chronological cross-validation.


In [ ]:
# season-level average of home implied probability
home_prob_by_season = (
    matches_odds
    .groupby("season")["b365_home_prob"]
    .mean()
    .reset_index()
    .sort_values("season")
)

home_prob_by_season


In [ ]:
fig, ax = plt.subplots(figsize=(9,5))

sns.lineplot(
    data=home_prob_by_season,
    x="season", y="b365_home_prob",
    marker="o", linewidth=2, color=PRIMARY_COLOR,
    ax=ax
)

ax.set_title("Average Implied Home Win Probability by Season (B365)")
ax.set_xlabel("Season")
ax.set_ylabel("Mean Implied Home Win Probability")

# for x, y in zip(home_prob_by_season["season"], home_prob_by_season["b365_home_prob"]):
#     ax.text(x, y + 0.003, f"{y:.2f}", ha="center", va="bottom", fontsize=9)

plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("results/EDA/home_implied_prob_drift_presentation.png", dpi=300)
plt.show()


## 11. Outcome vs Home Advantage Index (Odds-Based Baseline Pattern)

Using the derived *home advantage index*:

\\[
\text{home\_advantage\_index} = P(\text{home win}) - P(\text{away win})
\\]

we bin matches and estimate:

- empirical frequency of home wins in each bin,
- which visualizes how strongly this signal is related to outcomes.

This represents an **industry baseline** relationship that our model should aim to approximate.


In [ ]:
# Bin the index into quantiles
matches_odds["hai_bin"] = pd.qcut(
    matches_odds["home_advantage_index"],
    q=10,
    labels=False,
    duplicates="drop"
)

bin_stats = (
    matches_odds
    .groupby("hai_bin")
    .agg(
        hai_mean=("home_advantage_index", "mean"),
        home_win_rate=("match_result", lambda x: (x=="home_win").mean()),
        draw_rate=("match_result", lambda x: (x=="draw").mean()),
        away_win_rate=("match_result", lambda x: (x=="away_win").mean())
    )
    .reset_index()
)

bin_stats


In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

sns.lineplot(
    data=bin_stats,
    x="hai_mean",
    y="home_win_rate",
    marker="o",
    color=PRIMARY_COLOR,
    linewidth=2,
    label="Home win rate",
    ax=ax
)

sns.lineplot(
    data=bin_stats,
    x="hai_mean",
    y="away_win_rate",
    marker="o",
    color=SECONDARY_COLOR,
    linewidth=2,
    label="Away win rate",
    ax=ax
)

ax.set_title("Home/Away Win Rate vs Home Advantage Index (B365)")
ax.set_xlabel("Home Advantage Index (P_home − P_away)")
ax.set_ylabel("Empirical Win Rate")

ax.legend()

plt.tight_layout()
plt.savefig("results/EDA/home_advantage_index_vs_outcome_presentation.png", dpi=300)
plt.show()


## 12. Correlation Between Odds-Based Features

We inspect correlations between:

- implied home/draw/away probabilities,
- and the home advantage index.

This helps us understand redundancy and structure in the bookmaker baseline features.


In [ ]:
odds_feat_cols = [
    "b365_home_prob",
    "b365_draw_prob",
    "b365_away_prob",
    "home_advantage_index"
]

odds_corr = matches_odds[odds_feat_cols].corr()
odds_corr


In [ ]:
fig, ax = plt.subplots(figsize=(5,4))

sns.heatmap(
    odds_corr,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    vmin=-1, vmax=1,
    square=True,
    cbar_kws={"shrink": 0.8},
    annot_kws={"size": 9},
    ax=ax
)

ax.set_title("Correlation Between Odds-Based Features")

plt.tight_layout()
plt.savefig("results/EDA/odds_features_correlation_presentation.png", dpi=300)
plt.show()


## 13. Summary of EDA Findings

Key observations:

- **Match outcome distribution**:  
  - Home wins ≈ 45%, draws ≈ 25%, away wins ≈ 30%  
  - Mild class imbalance → use **Macro-F1** instead of simple accuracy.

- **League representation**:  
  - A few leagues (e.g., England, Spain, Germany) dominate match counts.  
  - Justifies **per-league metrics** and fairness auditing.

- **Missingness**:  
  - Line-up columns and betting odds show high missingness.  
  - Justifies **excluding line-ups and odds** in the v1 model to avoid noisy imputations and leakage.

- **Player attributes**:  
  - `overall_rating` is well-distributed (not constant, no extreme outliers).  
  - Strong correlations between overall rating, potential, and technical skills.

- **Temporal drift**:  
  - Average player rating evolves across years.  
  - Confirms the need for **chronological train-test splits** and **time-aware modeling**.

All plots and tables used here are saved under:

- `results/EDA/` (for GitHub + final report + model card).  
